In [1]:
# Let’s say you are trying to use a noisy, blurred, low resolution, heavily quantized video feed to estimate the x,y position of a ball.  The primary objective is to estimate the position of the ball as accurately as possible, and as quickly as possible!

# Here is what we know:
#     The field of view of the camera is dark (pixel intensity 0), except for a single ball with uniform brightness (pixel intensity 7) 
#     The ball is exactly 10 pixels in diameter, and has perfectly elastic collisions will the four “walls” bounding the square field of view of the camera
#     There are no external forces acting on the ball (no gravity, no air resistance)
#     The camera's CCD has 101x101 pixels, each with 3-bit intensity resolution [0,7] (greyscale)
#     The camera’s CCD is noisy. Each pixel has noise ~unif(0,5).  This noise is uncorrelated from pixel to pixel, and frame to frame.
#     The ball’s initial position is pixel [0,0] but its initial velocity vector is unknown. The maximum velocity is 7.5 pixels per frame.

# Here are the deliverables:
#     Python simulation of this system. Please comment your code thoroughly, as it will be reviewed in detail.  If you use any open source code, please include citations.
#     A dozen slides containing:
#         Block diagram depicting your simulation structure
#         Why you chose the algorithms you did (what else did you consider?)
#         Plot of estimation accuracy vs frame count (does it converge? Any residual “steady state” error in the long run? Does performance depend on initial condition?)
#         Some analytical justification for the simulation results you are seeing (did simulation agree with analytical? If so, why? If not, why not?)
#         Estimate of total processing resources required to run your system in real time (MAC operations/frame)
#         What would you do differently in your approach if you had more time?
#         What did you find most challenging about this project

In [16]:
# Imports (Run once)
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [17]:
# This function adds uniformly distributed noise to each frame and quantizes it
def add_noise(x, lim_low=0.0, lim_high=5.0, bitdepth=3,rs=np.random.RandomState(seed=42)):
    
    b = rs.uniform(low=lim_low, high=lim_high, size=x.shape) + x # Add uniform noise
    adu = b.astype(np.int) # Convert to discrete numbers
    max_adu = np.int(2**bitdepth - 1)
    adu[adu>max_adu] = max_adu # Models pixel saturation
    

In [12]:
vel = [random.uniform(0.1,7.5) for i in range(2)] # generate a vector with a random x and y velocity
pos = [0,0] # Initial Position of the ball is at (0,0)